In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from lxml import etree
from tqdm import tqdm
import re

from nlppln.utils import get_files

in_files = get_files('/home/jvdzwaan/Downloads/dbnl_ocr/xml/')
print(len(in_files))

In [ ]:
import os
from tqdm import tqdm_notebook as tqdm
from codecs import open
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

def replace_entities(fname, out_dir):
    with open(fname, encoding='utf8') as f:
        text = f.read()
    text = text.replace('&nbsp;', '&#160;')
    text = text.replace('&ldquo;', '&#8220;')
    text = text.replace('&rdquo;', '&#8221;')
    text = text.replace('&lsquo;', '&#8216;')   # left single quote
    text = text.replace('&rsquo;', '&#8217;')   # right single quote
    text = text.replace('&copy;', '&#169;')
    text = text.replace('&fnof;', '&#192;')     # f-sign (guilders)
    text = text.replace('&rarr;', '&#8594;')    # rightwards arrow
    text = text.replace('&Euml;', '&#203;')     # capital E, dieresis or umlaut mark
    text = text.replace('&Uuml;', '&#220;')     # capital U, dieresis or umlaut mark
    text = text.replace('&Auml;', '&#196;')     # capital A, dieresis or umlaut mark
    text = text.replace('&Ouml;', '&#214;')     # capital O, dieresis or umlaut mark
    text = text.replace('&Egrave;', '&#353;')     # capital E, grave accent
    text = text.replace('&scaron;', '&#200;')     # latin small letter s with caron
    text = text.replace('&yacute;', '&#253;')     # y, acute accent
    text = text.replace('&otilde;', '&#245;')   # small o, tilde
    text = text.replace('&atilde;', '&#227;')   # small a, tilde
    text = text.replace('&ntilde;', '&#241;')   # small n, tilde
    text = text.replace('&plusmn;', '&#177;')   # +/-
    text = text.replace('&times;', '&#215;')    # multiply sign
    text = text.replace('&sect;', '&#167;')    # section (?)
    text = text.replace('&pi;', '&#960;')    # pi
    text = text.replace('&Epsilon;', '&#917;')    # greek capital letter epsilon
    text = text.replace('&rho;', '&#961;')    # greek small letter rho
    text = text.replace('&omega;', '&#969;')    # greek small letter omega
    text = text.replace('&tau;', '&#964;')    # greek small letter tau
    text = text.replace('&omicron;', '&#959;')    # greek small letter omicron
    text = text.replace('&alpha;', '&#945;')    # greek small letter alpha
    text = text.replace('&iota;', '&#953;')    # greek small letter iota
    text = text.replace('&gamma;', '&#947;')    # greek small letter gamma
    text = text.replace('&nu;', '&#957;')    # greek small letter nu
    text = text.replace('&sigmaf;', '&#962;')    # greek small letter final sigma
    text = text.replace('&upsilon;', '&#965;')    # greek small letter upsilon
    text = text.replace('&kappa;', '&#954;')    # greek small letter kappa
    text = text.replace('&mu;', '&#956;')    # greek small letter mu
    text = text.replace('&lambda;', '&#955;')    # greek small letter lambda
    text = text.replace('&epsilon;', '&#949;')    # greek small letter epsilon
    text = text.replace('&eta;', '&#951;')    # greek small letter eta
    text = text.replace('&delta;', '&#948;')    # greek small letter delta
    text = text.replace('&theta;', '&#952;')    # greek small letter theta
    text = text.replace('&phi;', '&#966;')    # greek small letter phi
    text = text.replace('&zeta;', '&#950;')    # greek small letter zeta
    text = text.replace('&sigma;', '&#963;')    # greek small letter sigma
    text = text.replace('&beta;', '&#946;')    # greek small letter beta
    text = text.replace('&chi;', '&#967;')    # greek small letter chi
    text = text.replace('&Tau;', '&#932;')    # greek capital letter tau
    text = text.replace('&Mu;', '&#924;')    # greek capital letter mu
    text = text.replace('&Eta;', '&#919;')    # greek capital letter eta
    text = text.replace('&Omicron;', '&#927;')    # greek capital letter omicron
    text = text.replace('&Delta;', '&#916;')    # greek small letter delta
    
     
    out_file = out_file_name(out_dir, fname)
    #print(out_file)
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(text)
    
    # Test whether result is valid xml
    #root = etree.parse(out_file)

out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-entities/'
create_dirs(out_dir)
#for in_file in tqdm(['/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-entities/ypes001petr01_01.xml']):
for in_file in tqdm(in_files):
    replace_entities(in_file, out_dir)

In [ ]:
%%time
# remove empty files
in_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-entities/'
!python -m nlppln.commands.delete_empty_files {in_dir} -o {in_dir}

In [ ]:
out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-entities/'
in_files = get_files(out_dir)
print(len(in_files))

In [ ]:
print(in_files[35])

In [ ]:
%%time
# extract notes from xml
# save notes to file (one note per line)
# TOO SLOW

import os
import re
from tqdm import tqdm_notebook as tqdm
from codecs import open
from bs4 import BeautifulSoup

from nlppln.utils import out_file_name, create_dirs

def extract_notes(in_file, out_dir, notes_dir):
    with open(in_file) as f:
        soup = BeautifulSoup(f, 'xml')
    
    num_notes = 0
    notes_file = out_file_name(notes_dir, in_file, ext='txt')
    
    with open(notes_file, 'w', encoding='utf-8') as f:
    
        notes = soup.find_all('note')
        for note in notes:
            n = note.extract()
            txt = n.get_text()
            if len(txt) > 0:
                num_notes += 1
                f.write(txt)
                f.write('\n')
                
    #print('num_notes:', num_notes)
                
    heads = soup.find_all('head')
    for head in heads:
        m = re.match(r'\[.+\]', head.get_text())
        if m:
            head.extract()

    out_file = out_file_name(out_dir, in_file)
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(soup.prettify())
    
out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-notes-and-headers/'
notes_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/notes/'

create_dirs(out_dir)
create_dirs(notes_dir)

#for in_file in ('/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-entities/_aio001jver01_01.xml', ):
for in_file in tqdm(in_files):
    extract_notes(in_file, out_dir, notes_dir)

In [ ]:
%%time
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

def process_xml(fname, out_dir, notes_dir):
    root = etree.parse(fname)
    #root = tree.getroot()
    #print(root.tag)
    #print(root.getroot().tag)
    
    notes = root.iter('note')
    notes_text = []
    num_notes = 0
    
    
    for note in notes:
        #print(note.getparent())
        parent = note.getparent()
        if parent.tag != 'notesStmt':
            num_notes += 1
            txt = ''.join(note.itertext())
            #print(txt)
            if len(txt.strip()) > 0:
                notes_text.append(txt.replace('\n', ' '))
    #print('num_notes:', num_notes)
    
    # because of nested notes, we need to iterate again for removing notes
    # we also need to reverse the notes list, to remove the nested notes before their parents
    # otherwise some notes will be left
    notes = [n for n in root.iter('note')]
    notes.reverse()
    for note in notes:
        note.getparent().remove(note)
    
    #print(len(notes_text))
    if len(notes_text) > 0:
        notes_file = out_file_name(notes_dir, in_file, ext='txt')
        with open(notes_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(notes_text))

    heads = root.iter('head')
    for head in heads:
        #print(head.text)
        m = re.match(r'\[.+\]', ''.join(head.itertext()))
        if m:
            #print('deleting')
            head.getparent().remove(head)
            
    out_file = out_file_name(out_dir, in_file)
    #print(out_file)
    with open(out_file, 'wb') as f:
        f.write(etree.tostring(root, pretty_print=True, encoding='utf-8'))


out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-notes-and-headers/'
notes_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/notes/'

create_dirs(out_dir)
create_dirs(notes_dir)

#for in_file in ('/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-entities/_aio001jver01_01.xml', ):
for in_file in tqdm(in_files):
    process_xml(in_file, out_dir, notes_dir)

In [ ]:
in_dir = out_dir

in_files = get_files(in_dir)
print(len(in_files))

In [ ]:
%%time
from nlppln.utils import create_dirs, out_file_name

def tei2txt(fname):
    root = etree.parse(fname)
    body = root.find('text').find('body')
    #print(body)
    txt = ''.join(body.itertext())
    return txt

in_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/xml-without-notes-and-headers/'
out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/gs-unfiltered'

create_dirs(out_dir)

in_files = get_files(in_dir)

for in_file in tqdm(in_files):
    txt = tei2txt(in_file)
    out_file = out_file_name(fname=in_file, out_dir=out_dir, ext='txt')
    print(out_file)
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(txt)
